In [ ]:
from sklearn.linear_model import LogisticRegression

from obp.dataset import (
    SyntheticBanditDataset,
    logistic_reward_function,
    linear_behavior_policy,
)

from obp.policy import IPWLearner, Random
from obp.ope import (
    OffPolicyEvaluation,
    RegressionModel,
    InverseProbabilityWeighting as IPS,
    DoublyRobust as DR,
)

## 人工データの生成

p.87

- `n_rounds`: データ数
- `n_actions`: 意思決定モデルが選択できる行動の数
- `context`: 各データに対する特徴量
- `action_context`: 行動を表現するone-hotベクトル
- `action`: 過去の意思決定モデル`pi_b`によって選択された行動
- `reward`: 過去の意思決定モデル`pi_b`によって選択された行動に対する報酬
- `expected_reward`: 各行動に対する報酬の期待値
- `pscore`: 過去の意思決定モデル`pi_b`による行動選択確率

In [ ]:
dataset = SyntheticBanditDataset(
    n_actions=3,
    dim_context=3,
    reward_function=logistic_reward_function,
    behavior_policy_function=linear_behavior_policy,
    random_state=12345,
)

In [ ]:
training_data = dataset.obtain_batch_bandit_feedback(n_rounds=10000)
validation_data = dataset.obtain_batch_bandit_feedback(n_rounds=10000)

In [ ]:
training_data

In [ ]:
training_data["action_context"].tolist()

In [ ]:
set(training_data["reward"].tolist())

In [ ]:
validation_data

## 意思決定モデルの学習

p.90

In [ ]:
ipw_learner = IPWLearner(
    n_actions=dataset.n_actions,
    base_classifier=LogisticRegression(C=100, random_state=12345),
)

In [ ]:
ipw_learner.fit(
    context=training_data["context"],
    action=training_data["action"],
    reward=training_data["reward"],
    pscore=training_data["pscore"],
)

In [ ]:
action_choice_by_ipw_learner = ipw_learner.predict(
    context=validation_data["context"]
)

In [ ]:
# 比較のためにランダムに行動を選択する
random = Random(n_actions=dataset.n_actions, random_state=12345)

# バリデーションデータに対して行動選択確率を計算する -> これはランダムなので、行動選択確率は一様分布
action_choice_by_random = random.compute_batch_action_dist(
    n_rounds=validation_data["n_rounds"]
)

## 意思決定モデルの性能を評価する

p.92

In [ ]:
# DR推定量を用いるために必要な目的変数予測モデルを構築する
# opeモジュールに実装されているRegressionModelに好みの機械学習モデルを渡すことで、目的変数予測モデルを構築できる
regression_model = RegressionModel(
    n_actions=dataset.n_actions,
    base_model=LogisticRegression(C=100, random_state=12345),
)

# fit_predictメソッドを用いることで、構築した目的変数予測モデルを用いて、バリデーションデータに対する期待報酬を推定できる
estimated_rewards_by_reg_model = regression_model.fit_predict(
    context=validation_data["context"],
    action=validation_data["action"],
    reward=validation_data["reward"],
    n_folds=3,
    random_state=12345,
)